## Test sur un code orphanet particulier

In [19]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

browser = webdriver.Firefox()
browser.get('https://www.orpha.net/consor/cgi-bin/Disease.php?lng=FR')
browser.find_element_by_id('ORPHA').click()
element = browser.find_element_by_id("autocomplete_diseaseGroup")  # Find the search box
browser.find_element_by_id('autocomplete_diseaseGroup').clear()
element.send_keys("289465" + Keys.RETURN)
browser.implicitly_wait(1)
lib = browser.find_element_by_class_name('oneResult').text
print(lib)


ORPHA:289465  Adermatoglyphie congénitale isolée
Plus d'informations
Synonyme(s) : Absence congénitale d'empreintes digitales ; Absence congénitale de dermatoglyphes


In [20]:
browser.close()

## Lancement

In [21]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from pandas import read_csv
import selenium.webdriver.support.ui as ui


In [22]:
#browser.close()
browser = webdriver.Firefox()
# browser.get('https://www.orpha.net/consor/cgi-bin/Disease.php?lng=FR')

## Fonction pour faire la requête code par code

In [23]:

def have_libelle(code, waiting = 0.8):
    # wait = ui.WebDriverWait(browser,0.8)
    browser.get('https://www.orpha.net/consor/cgi-bin/Disease.php?lng=FR')
    browser.find_element_by_id('ORPHA').click()
    element = browser.find_element_by_id("autocomplete_diseaseGroup")  # Find the search box
    browser.find_element_by_id('autocomplete_diseaseGroup').clear()
    element.send_keys(code + Keys.RETURN)
    browser.implicitly_wait(waiting * 5)
    
    nope = browser.find_elements_by_class_name('missingInfo')
    
    if nope:
        lib = ''
        lib_infos = ''
    else:
        # Informations globales
        #wait.until(lambda browser: browser.find_element_by_class_name('oneResult'))
        lib = browser.find_element_by_class_name('oneResult').text
        browser.find_element_by_class_name('btn-lnk').click()
        browser.implicitly_wait(waiting * 2)
        # Informations détaillées : (synonymes, prévalence, hérédité, âge, code CIM, 
        # OMIM, UMLS, MEsH, GARD, MedDRA)
        # wait.until(lambda browser: browser.find_element_by_class_name('idData'))
        lib_infos = browser.find_element_by_class_name('idData').text
    return lib + '\n--\nDétails\n--\n' + lib_infos


In [24]:
# Test de la fonction
have_libelle('289465', waiting = 2)

"ORPHA:289465  Adermatoglyphie congénitale isolée\nPlus d'informations\nSynonyme(s) : Absence congénitale d'empreintes digitales ; Absence congénitale de dermatoglyphes\n--\nDétails\n--\nSynonyme(s) :\nAbsence congénitale d'empreintes digitales\nAbsence congénitale de dermatoglyphes\nPrévalence : Inconnu\nHérédité : Autosomique dominante \nÂge d'apparition : Petite enfance, Néonatal\nCIM-10 : Q82.8\nOMIM : 136000\nUMLS : -\nMeSH : -\nGARD : 12550\nMedDRA : -"

## Lire les codes orphanet présents dans le pdf (extraits avec pdftools et R)

In [25]:
import os
import numpy as np
u = read_csv('~/Documents/python_/orpha/data/liste_codes_janvier_2018.txt')
u.head()

,liste_codes
0,289465
1,210122
2,48818
3,2008
4,930


In [32]:
v = os.listdir('res')
uu = u['liste_codes'].tolist()
w = np.setdiff1d(uu,v)

n0 = 0
n = len(w)
print(n)
print(len(v))
waitt = 1

1148
5889


In [31]:
for i in range(n0, n):
    # print(str(i) + ' : ' + str(u['liste_codes'][i]))
    r = have_libelle(str(w[i]), waitt)
    if r != '\n--\nDétails\n--\n':
        with open('res/' + str(w[i]), "w") as output:
            output.write(r + '\n')
        output.close()


KeyboardInterrupt: 

In [ ]:
# Liste résultat 
# print(v)

In [ ]:
# On écrit le résultat dans un fichier (méthode 2)
# import csv

# with open('result_1.csv', "w") as output:
#     writer = csv.writer(output, lineterminator='\n')
#     for val in v:
#         writer.writerow([val])    
